In [ ]:
import syft_client as sc
import syft_serve as ss
from loguru import logger

In [ ]:
client = sc.login("rasswanth@openmined.org")

In [ ]:
ss.servers

In [ ]:

def create_job_runner_functions(syftbox_folder: str, poll_interval: int = 1):
    """Factory that returns syft-serve compatible functions"""

    def start_runner():
        """Function compatible with syft-serve"""
        import threading
        import tempfile
        import os
        from pathlib import Path

        # Use a file-based lock
        lock_file = Path(tempfile.gettempdir()) / f"job_runner_{abs(hash(syftbox_folder))}.lock"

        # Try to create lock file atomically
        try:
            fd = os.open(lock_file, os.O_CREAT | os.O_EXCL | os.O_WRONLY)
            os.write(fd, str(os.getpid()).encode())
            os.close(fd)
        except OSError:
            # Lock file exists, check if process is still running
            try:
                with open(lock_file, 'r') as f:
                    pid = int(f.read().strip())
                    os.kill(pid, 0)  # Check if process exists
                return {"status": "already_running"}
            except (ValueError, ProcessLookupError, FileNotFoundError):
                # Stale lock file, remove it
                lock_file.unlink(missing_ok=True)
                try:
                  fd = os.open(lock_file, os.O_CREAT | os.O_EXCL | os.O_WRONLY)
                  os.write(fd, str(os.getpid()).encode())
                  os.close(fd)
                except OSError:
                    return {"status": "error", "message": "Failed to acquire lock"}
                
        def _main_job_runner():
            print("Hello from job runner!")

        # Start the job
        thread = threading.Thread(target=_main_job_runner, daemon=True)
        thread.start()

        return {
            "syftbox_folder": syftbox_folder,
            "poll_interval": poll_interval
        }

    return start_runner

# Create the functions
start_fn = create_job_runner_functions("/path/to/syftbox", poll_interval=1)


In [ ]:
# Cell 2: Create syft-serve server
server = ss.create(
    name="job_runner_test",
    endpoints={
        "/start": start_fn,
    },
    dependencies=[],
)


In [ ]:
server

In [ ]:
ss.servers

In [ ]:
import requests
res = requests.get(f"{server.url}/start")

res.raise_for_status()

In [ ]:
res.content

In [ ]:
ss.servers[0]

In [ ]:
ss.terminate_all()

In [ ]:
def hello():
    return "Hello, World!"

In [ ]:
server = ss.create("hello_server", endpoints={"/": hello}, dependencies=[])

In [ ]:
server